<div style="font-size: 15px"><pre class="tab"><b>Name-</b>Haardik Gilhotra          <b>Roll no.-</b>R177219086          <b>Sap Id-</b>500076610          <b>Batch</b>-3 AI&ML</pre>
<hr>
<div style="font-size: 30px"><b><u><center>EXPERIMENT - 3</center></u></b>
    <div style="font-size: 20px"><b>Aim: Wine Quality Prediction</b>
<!--jupyter-nbconvert --to PDFviaHTML winequality.ipynb-->

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.linear_model import LinearRegression as lr
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as acc, confusion_matrix as cf
import matplotlib.pyplot as plt

In [2]:
# Reading dataset
wines = pd.read_csv("winequality.csv")
wines.head(2)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6


In [3]:
# Checking if dataset has any nan values
len(wines[wines.isna().any(axis=1)])

34

In [4]:
# Filling nan values with the mean 
for i in wines:
    if i != "type":
        wines[i].fillna(value=wines[i].mean(), inplace=True)

In [5]:
# Checking if nan values have been removed or not
len(wines[wines.isna().any(axis=1)])

0

In [6]:
target = wines.drop('type', axis = 1)
cols = (list(target.columns))
cols.remove('quality')
target.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6


In [7]:
def remove_outliers(col,target):
    data = target
    data["z_" + col] = (data[col] - data[col].mean()) / data[col].std()
    data = data[np.abs(data["z_" + col])<3]
    return data
for col in cols:
    print("Removing outliers in ",col)
    target = remove_outliers(col, target)
print("Total outliers removed: ", wines.shape[0] - target.shape[0])

Removing outliers in  fixed acidity
Removing outliers in  volatile acidity
Removing outliers in  citric acid
Removing outliers in  residual sugar
Removing outliers in  chlorides
Removing outliers in  free sulfur dioxide
Removing outliers in  total sulfur dioxide
Removing outliers in  density
Removing outliers in  pH
Removing outliers in  sulphates
Removing outliers in  alcohol
Total outliers removed:  558


In [8]:
target = target.drop(target.loc[:,"z_fixed acidity":], axis = 1)
target.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


In [9]:
# Defining the dependent and independent variables
x = target.drop(['quality'],axis = 1).values
y = target['quality']
x = ss().fit_transform(x)
y = (y > 6).astype(np.uint8)

In [10]:
# Splitting the dataset into training and testing data
x_train, x_test, y_train, y_test = tts(x,y,test_size = 0.20)

In [11]:
# Loading the models
models = {
    "Linear Regression": lr(),
    "SVC": SVC(),
    "K Nearest Neighbours": knn(n_neighbors=10),
    "Random Forest Classifier": rf()
}

In [12]:
# Training and printing the accuracy of the model
for name, model in models.items():
    model.fit(x_train,y_train)
    print(name)
    print("Training accuracy: {:.2%}".format(model.score(x_train,y_train)))
    print("Testing accuracy: {:.2%}\n".format(model.score(x_test,y_test)))

Linear Regression
Training accuracy: 18.94%
Testing accuracy: 20.65%

SVC
Training accuracy: 84.47%
Testing accuracy: 81.90%

K Nearest Neighbours
Training accuracy: 85.94%
Testing accuracy: 83.16%

Random Forest Classifier
Training accuracy: 100.00%
Testing accuracy: 88.72%



In [13]:
# Evaluating the model
for name, model in models.items():
    print(name)
    y_pred = model.predict(x_test)
    y_pred = (np.rint(y_pred)).astype(np.uint)
    print("Confusion Matrix: \n",cf(y_test, y_pred))
    print("Accuracy: {:.2%}\n".format(acc(y_test, y_pred)))

Linear Regression
Confusion Matrix: 
 [[931  18]
 [201  38]]
Accuracy: 81.57%

SVC
Confusion Matrix: 
 [[918  31]
 [184  55]]
Accuracy: 81.90%

K Nearest Neighbours
Confusion Matrix: 
 [[903  46]
 [154  85]]
Accuracy: 83.16%

Random Forest Classifier
Confusion Matrix: 
 [[921  28]
 [106 133]]
Accuracy: 88.72%

